In [8]:
from typing import TypedDict, Literal
from dotenv import load_dotenv
from langgraph.types import Command
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

load_dotenv()

llm = init_chat_model("openai:gpt-4o-mini")

In [9]:
class State(TypedDict):
    document: str
    summary: str
    sentiment: str
    key_points: str
    recommendation: str
    final_analysis: str

In [10]:
def get_summary(state: State):
    response = llm.invoke(
        f"이 문서에 대한 3-5 문장 요약을 작성하세요. {state['document']}"
    )

    return {
        "summary": response.content,
    }


def get_sentiment(state: State):
    response = llm.invoke(
        f"이 문서의 감정적 분위기와 어조를 평가하세요. {state['document']}"
    )

    return {
        "sentiment": response.content,
    }


def get_key_points(state: State):
    response = llm.invoke(
        f"이 문서에서 가장 중요한 5가지 핵심 사항을 추출하세요. {state['document']}"
    )

    return {
        "key_points": response.content,
    }


def get_recommendation(state: State):
    response = llm.invoke(
        f"이 문서를 기반해서 다음 단계를 위한 세 가지 추천사항을 제시하세요. {state['document']}"
    )

    return {
        "recommendation": response.content,
    }


def get_final_analysis(state: State):
    response = llm.invoke(
        f"""
        다음 보고서에 대한 분석을 해주세요.
        
        문서 분석 리포트
        ====================
        요약: {state["summary"]}

        감정 분석: {state["sentiment"]}
        
        핵심 사항: {state["key_points"]}
        
        추천 사항: {state["recommendation"]}                  
        """
    )

    return {
        "final_analysis": response.content,
    }


In [11]:
graph_builder = StateGraph(State)

graph_builder.add_node("get_summary", get_summary)
graph_builder.add_node("get_sentiment", get_sentiment)
graph_builder.add_node("get_key_points", get_key_points)
graph_builder.add_node("get_recommendation", get_recommendation)
graph_builder.add_node("get_final_analysis", get_final_analysis)

graph_builder.add_edge(START, "get_summary")
graph_builder.add_edge(START, "get_sentiment")
graph_builder.add_edge(START, "get_key_points")
graph_builder.add_edge(START, "get_recommendation")

graph_builder.add_edge("get_summary", "get_final_analysis")
graph_builder.add_edge("get_sentiment", "get_final_analysis")
graph_builder.add_edge("get_key_points", "get_final_analysis")
graph_builder.add_edge("get_recommendation", "get_final_analysis")
graph_builder.add_edge("get_final_analysis", END)


graph = graph_builder.compile()

In [12]:
# graph

In [13]:
with open("fed_transcript.md", "r", encoding="utf-8") as f:
    document = f.read()

for chunk in graph.stream(
    {"document": document},
    stream_mode="updates",
):
    print(chunk, "\n")

{'get_summary': {'summary': '이 문서는 연방준비제도(Fed)가 최대 고용과 안정된 가격이라는 두 가지 목표를 달성하기 위해 금리를 0.25% 인하하기로 결정했다는 내용을 다루고 있습니다. 현재 실업률은 4.3%로 약간 상승하였으며, 최근 소비자 지출이 둔화되면서 경제 성장도 1.5%로 감소했습니다. 물가 상승률은 지난 1년 동안 2.7%로 여전히 높은 상태이며, 향후 2% 목표를 달성하기 위한 실질적인 조치를 취하고 있음을 강조합니다. 노동 시장의 약세와 인플레이션 위험이 상존하는 상황에서는 두 목표 간의 균형을 유지해야 한다고 대통령은 언급하였습니다.'}} n
{'get_recommendation': {'recommendation': "Based on the content of the document you provided, here are three recommendations for the next steps regarding monetary policy and economic strategy:\n\n1. **Focus on Adaptive Monetary Policy**: Given the uncertainty in the labor market and the mixed signals regarding inflation, it is crucial to adopt a more adaptive monetary policy approach. The Federal Open Market Committee (FOMC) should closely monitor economic indicators and be prepared to adjust interest rates proactively, as necessary, to balance the dual mandate of maximum employment and stable prices. This might include further rate cuts or pauses in the reduct